# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f6af8ed9ca0>
├── 'a' --> tensor([[-0.0243,  0.6236,  0.4430],
│                   [ 1.2774, -0.0777, -0.0534]])
└── 'x' --> <FastTreeValue 0x7f6af8ed9940>
    └── 'c' --> tensor([[ 1.2640, -0.7245,  0.1980,  0.2504],
                        [ 0.1343, -0.3038, -0.0161, -0.8113],
                        [ 0.9927, -1.0041,  0.3425, -0.9669]])

In [4]:
t.a

tensor([[-0.0243,  0.6236,  0.4430],
        [ 1.2774, -0.0777, -0.0534]])

In [5]:
%timeit t.a

65.6 ns ± 0.0189 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f6af8ed9ca0>
├── 'a' --> tensor([[-1.6647, -0.1696,  1.2950],
│                   [ 1.1025, -0.4141,  0.7832]])
└── 'x' --> <FastTreeValue 0x7f6af8ed9940>
    └── 'c' --> tensor([[ 1.2640, -0.7245,  0.1980,  0.2504],
                        [ 0.1343, -0.3038, -0.0161, -0.8113],
                        [ 0.9927, -1.0041,  0.3425, -0.9669]])

In [7]:
%timeit t.a = new_value

63.4 ns ± 0.0627 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.0243,  0.6236,  0.4430],
               [ 1.2774, -0.0777, -0.0534]]),
    x: Batch(
           c: tensor([[ 1.2640, -0.7245,  0.1980,  0.2504],
                      [ 0.1343, -0.3038, -0.0161, -0.8113],
                      [ 0.9927, -1.0041,  0.3425, -0.9669]]),
       ),
)

In [10]:
b.a

tensor([[-0.0243,  0.6236,  0.4430],
        [ 1.2774, -0.0777, -0.0534]])

In [11]:
%timeit b.a

51.2 ns ± 0.0468 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.9257, -1.0713, -1.7909],
               [-0.5314,  0.0387, -0.8394]]),
    x: Batch(
           c: tensor([[ 1.2640, -0.7245,  0.1980,  0.2504],
                      [ 0.1343, -0.3038, -0.0161, -0.8113],
                      [ 0.9927, -1.0041,  0.3425, -0.9669]]),
       ),
)

In [13]:
%timeit b.a = new_value

480 ns ± 0.221 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

826 ns ± 0.109 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12 µs ± 27.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

134 µs ± 267 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 365 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f6acc464310>
├── 'a' --> tensor([[[-0.0243,  0.6236,  0.4430],
│                    [ 1.2774, -0.0777, -0.0534]],
│           
│                   [[-0.0243,  0.6236,  0.4430],
│                    [ 1.2774, -0.0777, -0.0534]],
│           
│                   [[-0.0243,  0.6236,  0.4430],
│                    [ 1.2774, -0.0777, -0.0534]],
│           
│                   [[-0.0243,  0.6236,  0.4430],
│                    [ 1.2774, -0.0777, -0.0534]],
│           
│                   [[-0.0243,  0.6236,  0.4430],
│                    [ 1.2774, -0.0777, -0.0534]],
│           
│                   [[-0.0243,  0.6236,  0.4430],
│                    [ 1.2774, -0.0777, -0.0534]],
│           
│                   [[-0.0243,  0.6236,  0.4430],
│                    [ 1.2774, -0.0777, -0.0534]],
│           
│                   [[-0.0243,  0.6236,  0.4430],
│                    [ 1.2774, -0.0777, -0.0534]]])
└── 'x' --> <FastTreeValue 0x7f6acc464610>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.6 µs ± 61.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f6a41133f70>
├── 'a' --> tensor([[-0.0243,  0.6236,  0.4430],
│                   [ 1.2774, -0.0777, -0.0534],
│                   [-0.0243,  0.6236,  0.4430],
│                   [ 1.2774, -0.0777, -0.0534],
│                   [-0.0243,  0.6236,  0.4430],
│                   [ 1.2774, -0.0777, -0.0534],
│                   [-0.0243,  0.6236,  0.4430],
│                   [ 1.2774, -0.0777, -0.0534],
│                   [-0.0243,  0.6236,  0.4430],
│                   [ 1.2774, -0.0777, -0.0534],
│                   [-0.0243,  0.6236,  0.4430],
│                   [ 1.2774, -0.0777, -0.0534],
│                   [-0.0243,  0.6236,  0.4430],
│                   [ 1.2774, -0.0777, -0.0534],
│                   [-0.0243,  0.6236,  0.4430],
│                   [ 1.2774, -0.0777, -0.0534]])
└── 'x' --> <FastTreeValue 0x7f6a41133dc0>
    └── 'c' --> tensor([[ 1.2640, -0.7245,  0.1980,  0.2504],
                        [ 0.1343, -0.3038, -0.0161, -0.8113],
                 

In [23]:
%timeit t_cat(trees)

29.9 µs ± 18.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

61.8 µs ± 56.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.0243,  0.6236,  0.4430],
                [ 1.2774, -0.0777, -0.0534]],
       
               [[-0.0243,  0.6236,  0.4430],
                [ 1.2774, -0.0777, -0.0534]],
       
               [[-0.0243,  0.6236,  0.4430],
                [ 1.2774, -0.0777, -0.0534]],
       
               [[-0.0243,  0.6236,  0.4430],
                [ 1.2774, -0.0777, -0.0534]],
       
               [[-0.0243,  0.6236,  0.4430],
                [ 1.2774, -0.0777, -0.0534]],
       
               [[-0.0243,  0.6236,  0.4430],
                [ 1.2774, -0.0777, -0.0534]],
       
               [[-0.0243,  0.6236,  0.4430],
                [ 1.2774, -0.0777, -0.0534]],
       
               [[-0.0243,  0.6236,  0.4430],
                [ 1.2774, -0.0777, -0.0534]]]),
    x: Batch(
           c: tensor([[[ 1.2640, -0.7245,  0.1980,  0.2504],
                       [ 0.1343, -0.3038, -0.0161, -0.8113],
                       [ 0.9927, -1.0041,  0.3425, -0.9669]],
         

In [26]:
%timeit Batch.stack(batches)

78.2 µs ± 331 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.0243,  0.6236,  0.4430],
               [ 1.2774, -0.0777, -0.0534],
               [-0.0243,  0.6236,  0.4430],
               [ 1.2774, -0.0777, -0.0534],
               [-0.0243,  0.6236,  0.4430],
               [ 1.2774, -0.0777, -0.0534],
               [-0.0243,  0.6236,  0.4430],
               [ 1.2774, -0.0777, -0.0534],
               [-0.0243,  0.6236,  0.4430],
               [ 1.2774, -0.0777, -0.0534],
               [-0.0243,  0.6236,  0.4430],
               [ 1.2774, -0.0777, -0.0534],
               [-0.0243,  0.6236,  0.4430],
               [ 1.2774, -0.0777, -0.0534],
               [-0.0243,  0.6236,  0.4430],
               [ 1.2774, -0.0777, -0.0534]]),
    x: Batch(
           c: tensor([[ 1.2640, -0.7245,  0.1980,  0.2504],
                      [ 0.1343, -0.3038, -0.0161, -0.8113],
                      [ 0.9927, -1.0041,  0.3425, -0.9669],
                      [ 1.2640, -0.7245,  0.1980,  0.2504],
                      [ 0.1343, -

In [28]:
%timeit Batch.cat(batches)

143 µs ± 371 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

304 µs ± 6.23 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
